This script shows how our data preprocessing steps that were done to generate and store the pre-processed data as pickle files to be used in fine-tuning our models as well as to visualize our raw data.

The raw data was downloaded from https://www.lib.umich.edu/database/link/10387 and can be found in ../data/social_explorer_raw

In that directory you will see several other datasets we were aspiring to include, but were not able to get to in time

In [22]:
import pandas as pd

Crime Data: 

Initially there were many fields representing different slices of the big picture, we felt that trimming this down to just 1 field would be better for fine-tuning. Towards this end we used two of the existing fields to calculate total violent and property crimes as a percent of the total population.

In [23]:
crime_df = pd.read_csv('../data/social_explorer_raw/crime.csv')
crime_df = crime_df.drop(index=0)
crime_df = crime_df.dropna()
crime_df = crime_df.reset_index(drop=True)

crime_df['Total Violent and Property Crimes '] = crime_df['Total Violent and Property Crimes '].astype('float')
crime_df['Total Population (2014 est.)'] = crime_df['Total Population (2014 est.)'].astype('float')
crime_df['Total Violent and Property Crimes'+' %'] = (crime_df['Total Violent and Property Crimes '] / crime_df['Total Population (2014 est.)']) * 100


# crime_df = crime_df[['Qualifying Name', 'Total Violent and Property Crimes %']].round(2)
crime_df = crime_df[['FIPS', 'Qualifying Name','Total Violent and Property Crimes %']].round(2)
crime_df.to_pickle('../data/social_explorer_processed/crime_df.pkl')
crime_df.head(100)

,FIPS,Qualifying Name,Total Violent and Property Crimes %
0,01001,"Autauga County, Alabama",3.19
1,01003,"Baldwin County, Alabama",2.22
2,01005,"Barbour County, Alabama",2.82
3,01007,"Bibb County, Alabama",1.11
4,01009,"Blount County, Alabama",2.42
...,...,...,...
95,04011,"Greenlee County, Arizona",2.00
96,04012,"La Paz County, Arizona",2.86
97,04013,"Maricopa County, Arizona",3.44
98,04015,"Mohave County, Arizona",3.32


Demographic data:

We shrink the demographic dataset to focus on population number of each ethnic group of the latest year(year 2020). We do not care about historical data as we are not interested in analyzing trends. We omit cols such as 2010 total population and ethic population of ethnic group in 2010.

Similar to crime data we calculated these as a percent of the total population.

In [29]:
demographic_df = pd.read_csv('../data/social_explorer_raw/Census2020_PL94_Redistricting_Data.csv')
demographic_df = demographic_df.drop(index=0)
demographic_df = demographic_df.reset_index(drop=True)

demographic_col_to_keep = ['FIPS', 'Qualifying Name','2020 Total Population','2020 Population Density', '2020 Housing Units',
       '2020 Occupied Units', '2020 Vacant Units',
       'White 2020', 'African American or Black 2020',
       'American Indian and Alaska Native 2020', 'Asian 2020',
       'Native Hawaiian and Other Pacific Islander 2020', 'Other 2020',
       'Two or More Races 2020',
       'Non-Hispanic 2020', 'Hispanic 2020']
demographic_df_filtered = demographic_df[demographic_col_to_keep]

columns_to_convert = [
       'White 2020', 'African American or Black 2020',
       'American Indian and Alaska Native 2020', 'Asian 2020',
       'Native Hawaiian and Other Pacific Islander 2020',
       'Other 2020', 'Two or More Races 2020',
       'Non-Hispanic 2020', 'Hispanic 2020'
]

demographic_df_filtered = demographic_df_filtered.dropna()
demographic_df_filtered['2020 Total Population'] = demographic_df_filtered['2020 Total Population'].astype('float')

for col in columns_to_convert:
    demographic_df_filtered[col] = demographic_df_filtered[col].astype('float')
    demographic_df_filtered[col+' %'] = (demographic_df_filtered[col] / demographic_df_filtered['2020 Total Population']) * 100

demographic_col_to_keep = ['FIPS','Qualifying Name','2020 Total Population','2020 Population Density',
       'White 2020 %', 'African American or Black 2020 %',
       'American Indian and Alaska Native 2020 %', 'Asian 2020 %',
       'Native Hawaiian and Other Pacific Islander 2020 %', 'Other 2020 %',
       'Two or More Races 2020 %', 'Hispanic 2020 %']
demographic_df = demographic_df_filtered[demographic_col_to_keep].round(2)
demographic_df.to_pickle('../data/social_explorer_processed/demographic_df.pkl')
demographic_df.head(100)

,FIPS,Qualifying Name,2020 Total Population,2020 Population Density,White 2020 %,African American or Black 2020 %,American Indian and Alaska Native 2020 %,Asian 2020 %,Native Hawaiian and Other Pacific Islander 2020 %,Other 2020 %,Two or More Races 2020 %,Hispanic 2020 %
0,01001,"Autauga County, Alabama",58805.0,98.9223582606555,71.69,19.46,0.37,1.50,0.06,1.55,5.37,3.60
1,01003,"Baldwin County, Alabama",231767.0,145.780443967561,81.72,7.86,0.68,0.89,0.06,2.30,6.48,5.47
2,01005,"Barbour County, Alabama",25223.0,28.5003067314037,44.87,47.31,0.46,0.46,0.00,4.12,2.78,5.99
3,01007,"Bibb County, Alabama",22293.0,35.8137988903938,74.26,19.80,0.27,0.14,0.04,2.09,3.40,3.32
4,01009,"Blount County, Alabama",59134.0,91.6961633730729,85.67,1.43,0.57,0.30,0.04,5.80,6.18,9.76
...,...,...,...,...,...,...,...,...,...,...,...,...
95,02282,"Yakutat City And Borough, Alaska",662.0,0.0868392610617368,37.61,0.60,38.52,4.83,0.15,0.45,17.82,4.38
96,02290,"Yukon-Koyukuk Census Area, Alaska",5343.0,0.0367025928718224,21.15,0.13,71.72,0.15,0.06,0.52,6.27,1.35
97,04001,"Apache County, Arizona",66021.0,5.89562221866851,22.53,0.28,71.21,0.47,0.03,1.86,3.62,5.85
98,04003,"Cochise County, Arizona",125447.0,20.2016182706572,65.15,3.81,1.20,2.16,0.39,10.11,17.17,33.97


Housing Data:

The latest year of housing data available on Social Explorer was year 2020. This dataset include property rental data such as fair market rent.

Needed to remove the last 5 characters from FIPS code

In [38]:
housing_df = pd.read_csv('../data/social_explorer_raw/Income_limits_and_fair_market_rent_2020.csv')
housing_df = housing_df.drop(index=0)
housing_df = housing_df.reset_index(drop=True)

housing_col_to_keep = ['FIPS', 'Qualifying Name', 'Median Family Income (5-year ACS)', 'Fair Market Rent for One Bedroom',
       'Fair Market Rent for Two Bedrooms',
       'Fair Market Rent for Three Bedrooms',
       'Fair Market Rent for Four Bedrooms']
housing_df = housing_df[housing_col_to_keep]
housing_df['FIPS'] = housing_df['FIPS'].str[:-5]
housing_df.to_pickle('../data/social_explorer_processed/housing_df.pkl')
housing_df.head(100)

,FIPS,Qualifying Name,Median Family Income (5-year ACS),Fair Market Rent for One Bedroom,Fair Market Rent for Two Bedrooms,Fair Market Rent for Three Bedrooms,Fair Market Rent for Four Bedrooms
0,01001,"Autauga County, Alabama",65700,702,830,1047,1425
1,01003,"Baldwin County, Alabama",81000,749,916,1251,1566
2,01005,"Barbour County, Alabama",43800,481,633,789,925
3,01007,"Bibb County, Alabama",73100,861,986,1291,1425
4,01009,"Blount County, Alabama",73100,861,986,1291,1425
...,...,...,...,...,...,...,...
95,02290,"Yukon-Koyukuk Census Area, Alaska",51400,717,816,1177,1433
96,04001,"Apache County, Arizona",43200,589,723,980,1117
97,04003,"Cochise County, Arizona",66300,658,857,1236,1505
98,04005,"Coconino County, Arizona",75200,1024,1266,1653,2003


Environment Data:

Similar to the previous dataframes we re-calculated these area metrics as percent of total land.

In [36]:
environment_df = pd.read_csv('../data/social_explorer_raw/Environmental_summaries_2011.csv')
environment_df = environment_df.drop(index=0)
environment_df = environment_df.reset_index(drop=True)

col_to_keep = ['FIPS', 'Qualifying Name', 'Total Area of Land Coverage', 'Area Of Land Cover Class 11, Open Water'
       , 'Area Of Land Cover Class 12, Perennial Ice/Snow', 'Area Of Land Cover Class 21, Developed, Open Space', 'Area Of Land Cover Class 22, Developed, Low Intensity', 'Area Of Land Cover Class 23, Developed, Medium Intensity', 'Area Of Land Cover Class 24, Developed, High Intensity', 'Area Of Land Cover Class 31, Barren Land (Rock/Sand/Clay)','Area Of Land Cover Class 41, Deciduous Forest','Area Of Land Cover Class 42, Evergreen Forest','Area Of Land Cover Class 43, Mixed Forest', 'Area Of Land Cover Class 52, Shrub/Scrub', 'Area Of Land Cover Class 71, Grassland/Herbaceous', 'Area Of Land Cover Class 81, Pasture/Hay', 'Area Of Land Cover Class 82, Cultivated Crops', 'Area Of Land Cover Class 90, Woody Wetlands', 'Area Of Land Cover Class 95, Emergent Herbaceous Wetlands']

environment_df = environment_df[col_to_keep]

environment_df = environment_df.fillna(0)
environment_df['Total Area of Land Coverage'] = environment_df['Total Area of Land Coverage'].astype('float')

columns_to_convert = ['Area Of Land Cover Class 11, Open Water'
       , 'Area Of Land Cover Class 12, Perennial Ice/Snow', 'Area Of Land Cover Class 21, Developed, Open Space', 'Area Of Land Cover Class 22, Developed, Low Intensity', 'Area Of Land Cover Class 23, Developed, Medium Intensity', 'Area Of Land Cover Class 24, Developed, High Intensity', 'Area Of Land Cover Class 31, Barren Land (Rock/Sand/Clay)','Area Of Land Cover Class 41, Deciduous Forest','Area Of Land Cover Class 42, Evergreen Forest','Area Of Land Cover Class 43, Mixed Forest', 'Area Of Land Cover Class 52, Shrub/Scrub', 'Area Of Land Cover Class 71, Grassland/Herbaceous', 'Area Of Land Cover Class 81, Pasture/Hay', 'Area Of Land Cover Class 82, Cultivated Crops', 'Area Of Land Cover Class 90, Woody Wetlands', 'Area Of Land Cover Class 95, Emergent Herbaceous Wetlands']

for col in columns_to_convert:
    environment_df[col] = environment_df[col].astype('float')
    environment_df[col+' %'] = (environment_df[col] / environment_df['Total Area of Land Coverage']) * 100

col_to_keep = ['FIPS', 'Qualifying Name', 'Total Area of Land Coverage', 'Area Of Land Cover Class 11, Open Water %'
       , 'Area Of Land Cover Class 12, Perennial Ice/Snow %', 'Area Of Land Cover Class 21, Developed, Open Space %', 'Area Of Land Cover Class 22, Developed, Low Intensity %', 'Area Of Land Cover Class 23, Developed, Medium Intensity %', 'Area Of Land Cover Class 24, Developed, High Intensity %', 'Area Of Land Cover Class 31, Barren Land (Rock/Sand/Clay) %','Area Of Land Cover Class 41, Deciduous Forest %','Area Of Land Cover Class 42, Evergreen Forest %','Area Of Land Cover Class 43, Mixed Forest %', 'Area Of Land Cover Class 52, Shrub/Scrub %', 'Area Of Land Cover Class 71, Grassland/Herbaceous %', 'Area Of Land Cover Class 81, Pasture/Hay %', 'Area Of Land Cover Class 82, Cultivated Crops %', 'Area Of Land Cover Class 90, Woody Wetlands %', 'Area Of Land Cover Class 95, Emergent Herbaceous Wetlands %']    

environment_df = environment_df[col_to_keep].round(2)

shortened_columns = {
    'Total Area of Land Coverage': 'Total Area',
    'Area Of Land Cover Class 11, Open Water %': 'Open Water %',
    'Area Of Land Cover Class 12, Perennial Ice/Snow %': 'Ice/Snow %',
    'Area Of Land Cover Class 21, Developed, Open Space %': 'Developed Open Space %',
    'Area Of Land Cover Class 22, Developed, Low Intensity %': 'Developed Low Intensity %',
    'Area Of Land Cover Class 23, Developed, Medium Intensity %': 'Developed Medium Intensity %',
    'Area Of Land Cover Class 24, Developed, High Intensity %': 'Developed High Intensity %',
    'Area Of Land Cover Class 31, Barren Land (Rock/Sand/Clay) %': 'Barren Land %',
    'Area Of Land Cover Class 41, Deciduous Forest %': 'Deciduous Forest %',
    'Area Of Land Cover Class 42, Evergreen Forest %': 'Evergreen Forest %',
    'Area Of Land Cover Class 43, Mixed Forest %': 'Mixed Forest %',
    'Area Of Land Cover Class 52, Shrub/Scrub %': 'Shrub/Scrub %',
    'Area Of Land Cover Class 71, Grassland/Herbaceous %': 'Grassland/Herbaceous %',
    'Area Of Land Cover Class 81, Pasture/Hay %': 'Pasture/Hay %',
    'Area Of Land Cover Class 82, Cultivated Crops %': 'Cultivated Crops %',
    'Area Of Land Cover Class 90, Woody Wetlands %': 'Woody Wetlands %',
    'Area Of Land Cover Class 95, Emergent Herbaceous Wetlands %': 'Emergent Herbaceous Wetlands %'
}

environment_df.rename(columns=shortened_columns, inplace=True)
environment_df = environment_df.fillna(0)
environment_df.to_pickle('../data/social_explorer_processed/environment_df.pkl')
environment_df.head(100)

,FIPS,Qualifying Name,Total Area,Open Water %,Ice/Snow %,Developed Open Space %,Developed Low Intensity %,Developed Medium Intensity %,Developed High Intensity %,Barren Land %,Deciduous Forest %,Evergreen Forest %,Mixed Forest %,Shrub/Scrub %,Grassland/Herbaceous %,Pasture/Hay %,Cultivated Crops %,Woody Wetlands %,Emergent Herbaceous Wetlands %
0,01001,"Autauga County, Alabama",1.565344e+09,1.52,0.0,4.08,1.34,0.43,0.11,0.27,17.39,18.69,12.90,17.71,1.70,12.10,6.15,5.18,0.44
1,01003,"Baldwin County, Alabama",4.224505e+09,2.25,0.0,5.03,1.95,0.76,0.22,0.55,0.50,24.49,2.28,13.16,5.31,5.94,9.85,25.50,2.20
2,01005,"Barbour County, Alabama",2.342655e+09,2.51,0.0,3.42,0.75,0.21,0.06,0.14,20.42,27.41,8.53,16.72,2.40,6.83,5.43,4.89,0.28
3,01007,"Bibb County, Alabama",1.621768e+09,0.73,0.0,3.49,0.42,0.15,0.03,0.30,27.11,29.16,13.17,11.70,3.74,5.23,0.56,4.02,0.19
4,01009,"Blount County, Alabama",1.685142e+09,1.00,0.0,4.69,1.30,0.31,0.08,0.46,33.13,10.22,6.11,5.98,4.56,26.89,4.53,0.69,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,04001,"Apache County, Arizona",2.905602e+10,0.11,0.0,0.48,0.15,0.03,0.00,2.49,0.16,27.38,0.00,53.17,15.68,0.04,0.03,0.12,0.15
96,04003,"Cochise County, Arizona",1.610516e+10,0.02,0.0,1.56,0.27,0.09,0.02,0.88,0.00,7.04,0.45,81.80,4.15,0.20,3.30,0.17,0.04
97,04005,"Coconino County, Arizona",4.833222e+10,0.16,0.0,0.51,0.16,0.07,0.02,4.60,0.05,25.75,0.00,58.26,10.20,0.01,0.01,0.14,0.06
98,04007,"Gila County, Arizona",1.242020e+10,0.67,0.0,0.56,0.25,0.07,0.01,0.36,0.00,37.51,0.00,59.92,0.35,0.01,0.01,0.14,0.15


The following cell is commented out and was just used for spot checking results of various data, we decided to leave it as it may be useful to others exploring this work in the future!

In [27]:
# demographic_df_filtered = demographic_df[demographic_df['Qualifying Name'] == 'Santa Clara']
# filtered_df_partial = demographic_df[demographic_df['Qualifying Name'].str.contains('Avery')]
# filtered_df_partial.head()